In [ ]:
%load_ext autoreload
%autoreload 2
import math

import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np
import phfit
import toolkit2lps
from pareto import discretize, paretoCDF, paretoICDF

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
plt.rcParams['figure.figsize'] = [14, 9]

In [ ]:
n = 100000
fig, [ax0, ax1] = plt.subplots(1, 2)
for a in [1.01, 1.05, 1.1, 1.2, 1.3, 1.4]:
    x = paretoICDF(n, a=a, lb=3e5, ub=1e9)
    ax0.plot(x, np.cumsum(x)/np.sum(x), label=f'a={a}')
    ax1.bar(str(a), np.mean(x))
ax0.legend()
ax0.set_ylabel('cumulative probability')
ax0.set_xlabel('x')
ax0.set_title('CDF')
ax1.set_xlabel('alpha')
ax1.set_ylabel('x')
ax1.set_title('Mean of distribution')
fig.suptitle(f'Pareto CDF and mean, n={len(u)} samples of x with equal probability')
fig.tight_layout()

In [ ]:
n = 250
fig, [ax0, ax1] = plt.subplots(1, 2)
for a in [1.2]:
    x = np.logspace(math.log(300*1024, 1024), 3, n, base=1024)
    x, p = paretoCDF(x, a=a)
    x, p = discretize(x, p)

    ax0.plot(x, np.cumsum(p*x)/np.sum(p*x), label=f'a={a}')
    ax0.axvline(x=55*1024**2, color='red', linestyle='--')
    ax0.axvline(x=90475270, color='blue', linestyle='--')
    ax1.bar(str(a), np.mean(p*x))

ax0.legend()
ax0.legend()
ax0.set_ylabel('cumulative probability')
ax0.set_xlabel('x')
ax0.set_title('CDF')
ax1.set_xlabel('alpha')
ax1.set_ylabel('x')
ax1.set_title('Mean of distribution')
fig.suptitle(f'Pareto ? and mean, n={n} samples exponentially spaced over x and discretized probability')

In [ ]:
with open('../tests/traffic.cue', 'w') as file:
    n = 250
    x = np.logspace(math.log(300*1024, 1024), 3, n, base=1024)
    x, p = paretoCDF(a=1.2, x=x)
    x, p = discretize(x, p)
    file.write(f'package tests\n\n_connsizespareto: [{",".join(map(str,x))}]\n_connprobpareto: [{",".join(map(str,p))}]')

In [ ]:
# with open('../PhFit/data', 'w') as file:
#     n = 250
#     x = np.logspace(math.log(300*1024, 1024), 3, n, base=1024)
#     x, p = paretoCDF(a=1.2, x=x)
#     # x, p = discretize(x, p)
#     for xi, pi in zip(x, p):
#         if 0 < pi < 1:
#             file.write(f'{xi} {pi}\n')

In [ ]:
n = 250
x = np.logspace(math.log(300*1024, 1024), 3, n, base=1024)
x, p = paretoCDF(x, a=1.2)
a, b = phfit.contfit(x, p, 4, 3)
toolkit2lps.optimal_threshold(a, b, body=4, tail=3, thmin=3e5, thmax=1e9, rho=0.875)